In [15]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

Building the LSTM encoding model for the HLA class and neoantigen sequences

In [16]:
def encode_seqCDR(seqCDR):
    encoding_list = []
    for i in range(len(seqCDR)):
        if seqCDR[i] == "*":
            encoding_list.append(np.zeros(5).reshape(1,5))
        elif seqCDR[i] == "_":
            # print("Error: seqCDR contains '_'")
            # encoding_list.append(np.zeros(5).reshape(1,5))
            return np.nan
        else:
            encoding_list.append(af.loc[seqCDR[i]].values.reshape(1,5))
    return np.array(encoding_list).reshape(1,-1)

af = pd.read_csv("~/data/project/pMHC-TCR/library/Atchley_factors.csv")
af.index = af["Amino acid"]
af.drop(columns=["Amino acid"], inplace=True)

In [19]:
class pMHC_datasets(nn.Module):
    '''
    This class is used to load the data for pMHC encoding
    '''
    def __init__(self, file_path) -> None:
        # super().__init__()
        df = self.basic_io(file_path)
        
        df["first3_pep_encode"] = df["first3_pep"].apply(lambda x: encode_seqCDR(x))
        df["last3_pep_encode"] = df["last3_pep"].apply(lambda x: encode_seqCDR(x))
        df = df.dropna()

        encoded_seq = np.concatenate(df["first3_pep_encode"].values)
        encoded_seq = np.concatenate((encoded_seq, np.concatenate(df["last3_pep_encode"].values)), axis=1)

        # one-hot encoding the HLA
        onehot_encoder = OneHotEncoder(sparse=False)
        one_hot_hla = onehot_encoder.fit_transform(df["HLA"].values.reshape(-1,1))
        # print(one_hot_hla.shape)

        self.data = np.concatenate((encoded_seq, one_hot_hla), axis=1)

    def basic_io(self, file_path):
        df = pd.read_csv(file_path, index_col=0)
        # we use the columns of HLA, NeoAA
        df = df[["HLA", "NeoAA"]]
        df["first3_pep"] = df["NeoAA"].apply(lambda x: x[:3])
        df["last3_pep"] = df["NeoAA"].apply(lambda x: x[-3:])
        # drop the duplicates
        df = df.drop_duplicates(subset=["first3_pep", "last3_pep"])
        # drop the rows with NaN
        df = df.dropna()
        return df
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

In [20]:
file_path = "~/data/project/data/seqData/20230228.csv"
pMHC_data = pMHC_datasets(file_path)

LSTM encoding model

class 